### 交叉熵损失函数在多分类任务中存在的问题

多分类任务中，神经网络会输出一个当前数据对应于各个类别的置信度分数，将这些分数通过softmax进行归一化处理，最终会得到当前数据属于每个类别的概率。

$$q_i=\frac{exp(z_i)}{\sum_{j=1}^K exp(z_j)}$$

然后计算交叉熵损失函数：

$$Loss = -\sum_{i=1}^K p_i logq_i$$
$$$$